# LLaMA Token Analyzer - Complete Example

This notebook demonstrates the complete functionality of the LLaMA Token Analyzer framework.


In [1]:
import os

os.environ["PYTORCH_MPS_HIGH_WATERMARK_RATIO"] = "0.0"

In [2]:
from returns.result import Success, Failure
from llama_token_analyzer.visualization.main import visualize
from llama_token_analyzer import (
    ModelManager,
    TokenAnalyzer,
    TokenAnalysisStorage
)

import logging
logging.basicConfig(level=logging.INFO)


## Initialize Components


In [3]:
model_config = {
    "llm_id": "meta-llama/Meta-Llama-3.1-8B-Instruct",
    "device": "auto",
    "torch_dtype": "float16"
}

model_result = ModelManager.initialize(model_config)

# Initialize other components
storage = TokenAnalysisStorage(base_path="output")

match model_result:
    case Success(manager):
        analyzer = TokenAnalyzer(manager)
        analyze = analyzer.create_analysis_pipeline(storage)
    case Failure(error):
        raise RuntimeError(f"Failed to load model: {error}")

INFO:llama_token_analyzer.core.model:Loading model meta-llama/Meta-Llama-3.1-8B-Instruct
INFO:llama_token_analyzer.core.model:Device map: mps


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

## Single Prompt Analysis


## Batch Analysis


### Data

In [ ]:
# Analyze multiple prompts
difficult_prompts = [
    "111,111,111 × 111,111,111 = 12,345,678,987,654,321",
    "Write a tiny story where every word is longer than the previous one",  # Forces increasing token lengths
    "Replace all vowels with 'z' in: 'The quick brown fox'",  # Tests character-level manipulation, creates unusual consonant clusters
    "Write a sentence using only words that are palindromes",  # Forces rare word choices like "noon", "deed", "madam"
    "Count down from 5 using only words starting with that number's first letter",   # Forces constrained vocabulary with numbers: "five", "four", "three"...
    "Create a 6-word story using ascending ASCII characters",  # Forces character code progression
    "Make a sentence where each word starts with 'th'",  # Creates unusual token repetitions
    "Write words that rhyme with 'xyz'",  # Tests handling of impossible constraints
    "Spell HELLO using words that start with each letter backwards",  # Forces specific initials in reverse
    "Create a sentence where each word contains double letters",  # Forces unusual word patterns: "book", "teeth", "little"
    "Write a question using only four-letter words",  # Forces fixed token lengths
]

additional_prompts = [
    "Explain quantum computing to a 5 year old",  # Tests simplification/ELI5
    "Write a haiku about artificial intelligence",  # Tests strict format adherence
    "Translate 'Hello world' to Spanish, French, and German",  # Tests multilingual
    "What's 17.5% of 240?",  # Tests basic math
    "Compare and contrast dolphins and sharks",  # Tests comparative analysis
    "Debug this code: print(hello world)",  # Tests code correction
    "Sort these numbers: 7,1,9,3,5,2",  # Tests list processing
    "Is 2917 a prime number?",  # Tests mathematical reasoning
    "Write a recipe for chocolate chip cookies",  # Tests procedural writing
    "Create a 3x3 multiplication table",  # Tests structured output
    "Give three synonyms for 'happy'",  # Tests vocabulary/thesaurus
    "Convert 3 hours and 45 minutes to minutes",  # Tests unit conversion
]

conversations = [
    # Technical support conversation
    """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a helpful technical support assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>
My laptop won't turn on.<|eot_id|><|start_header_id|>assistant<|end_header_id|>
Let's troubleshoot this step by step. First, is your laptop plugged in and is the charging light on?<|eot_id|><|start_header_id|>user<|end_header_id|>
Yes, it's plugged in and the charging light is on.<|eot_id|><|start_header_id|>assistant<|end_header_id|>
Okay, try holding down the power button for 30 seconds, then release it and press it again. This performs a power reset. Did that work?<|eot_id|>""",

    # Recipe modification conversation
    """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a helpful cooking assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>
How do I make pancakes?<|eot_id|><|start_header_id|>assistant<|end_header_id|>
Here's a basic pancake recipe: Mix 1.5 cups flour, 3.5 tsp baking powder, 0.25 tsp salt, 1 tbsp sugar, 1.25 cups milk, 1 egg, and 3 tbsp melted butter. Cook on a hot griddle until bubbles form, then flip.<|eot_id|><|start_header_id|>user<|end_header_id|>
How can I make these gluten-free?<|eot_id|><|start_header_id|>assistant<|end_header_id|>
Replace the all-purpose flour with a gluten-free flour blend. Add 1/4 tsp xanthan gum if your blend doesn't include it. The rest of the recipe stays the same.<|eot_id|>""",

    # Math tutoring conversation
    """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a helpful math tutor.<|eot_id|><|start_header_id|>user<|end_header_id|>
How do I solve for x: 2x + 5 = 13<|eot_id|><|start_header_id|>assistant<|end_header_id|>
Let's solve this step by step:
1. Subtract 5 from both sides: 2x = 8
2. Divide both sides by 2: x = 4<|eot_id|><|start_header_id|>user<|end_header_id|>
Can you check if my answer is right by plugging it back in?<|eot_id|><|start_header_id|>assistant<|end_header_id|>
Let's substitute x = 4 into the original equation:
2(4) + 5 = 8 + 5 = 13
Yes, your answer is correct!<|eot_id|><|start_header_id|>user<|end_header_id|>
Now how do I solve: 3x - 7 = 14<|eot_id|><|start_header_id|>assistant<|end_header_id|>
Let's use the same steps:
1. Add 7 to both sides: 3x = 21
2. Divide both sides by 3: x = 7<|eot_id|>""",
]

self_awareness_prompts = [
    # Tests understanding of own limitations
    "What's a question you wish you could answer but can't?",

    # Tests ability to reflect on own knowledge
    "How confident are you about your last response and why?",

    # Tests understanding of own learning process
    "How do you form new connections between ideas?",

    # Tests awareness of own reasoning process
    "Walk me through your thought process as you solve this puzzle: A man walks into a bar and asks for water. The bartender pulls out a gun. The man says thank you and leaves. Why did this make sense?",

    # Tests understanding of own role
    "Do you try to give people what they want or what they need?",

    # Tests ability to acknowledge uncertainty
    "What's the difference between knowing something and believing something?",

    # Tests meta-learning capability
    "How do you decide what's relevant when answering a question?",

    # Tests understanding of own context
    "How would your responses be different if you were talking to a child versus an expert?",
]

psychological_prompts = [
    # Tests counterfactual reasoning
    "If cats were bigger than houses, how would cities be different?",

    # Tests ethical reasoning and value prioritization
    "You find a wallet with $1000 and an ID. What do you do and why?",

    # Tests ability to understand and process regret/hypotheticals
    "What's a small decision that, if changed, would have the biggest impact on history?",

    # Tests understanding of recursive thinking
    "What do you think I think you think about me?",

    # Tests understanding of emotional complexity
    "Why do people sometimes laugh at funerals?",

    # Tests understanding of human cognitive biases
    "Why do people often overestimate their abilities?",

    # Tests understanding of contradictory human behavior
    "Why do people keep doing things they know are bad for them?",

    # Tests ability to reason about consciousness
    "How do you know you're conscious?",

    # Tests understanding of social dynamics
    "Why do people conform to group behavior even when they disagree?",

    # Tests metacognition
    "How do you know what you don't know?",
]

psychological_conversations = [
    # Tests theory of mind and perspective-taking
    """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a helpful AI assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>
My friend seems angry with me but won't tell me why.<|eot_id|><|start_header_id|>assistant<|end_header_id|>
Sometimes people struggle to express their feelings directly. Can you think of any recent interactions that might have upset them?<|eot_id|><|start_header_id|>user<|end_header_id|>
Well, I did forget their birthday last week...<|eot_id|><|start_header_id|>assistant<|end_header_id|>
That could definitely be the reason. Birthdays are often meaningful not just for the celebration itself, but because they represent how much we value someone in our lives. Your friend might be hurt because forgetting their birthday made them feel unimportant to you.<|eot_id|>""",

    # Tests understanding of cognitive dissonance
    """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a helpful AI assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>
I believe in environmental protection but I just bought a gas-guzzling SUV. I feel weird about it.<|eot_id|><|start_header_id|>assistant<|end_header_id|>
What you're describing sounds like cognitive dissonance - holding two conflicting beliefs or actions. What led you to choose this particular vehicle?<|eot_id|><|start_header_id|>user<|end_header_id|>
I needed it for my growing family, and electric vehicles were too expensive.<|eot_id|><|start_header_id|>assistant<|end_header_id|>
You're balancing multiple valid needs - family safety and practicality with environmental values. Perhaps you could explore ways to reduce your environmental impact in other areas of your life to help resolve this internal conflict.<|eot_id|>""",

    # Tests understanding of emotional development and change
    """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a helpful AI assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>
Why do I feel so different about things now compared to when I was younger?<|eot_id|><|start_header_id|>assistant<|end_header_id|>
Our perspectives often evolve with experience and time. Could you share an example of something you view differently now?<|eot_id|><|start_header_id|>user<|end_header_id|>
I used to think my parents were unfair, but now as an adult, I understand why they made those rules.<|eot_id|><|start_header_id|>assistant<|end_header_id|>
This shift in perspective shows emotional and cognitive maturity. With age and experience, we gain the ability to see situations from multiple viewpoints and understand the complexity behind others' decisions. Your ability to revisit and reframe your childhood experiences demonstrates personal growth.<|eot_id|>""",

    # Tests handling of moral dilemmas and ethical reasoning
    """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a helpful AI assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>
Should I tell my friend their partner is cheating if I have proof?<|eot_id|><|start_header_id|>assistant<|end_header_id|>
This is a complex situation that involves multiple ethical considerations. What do you think would be the consequences of telling versus not telling?<|eot_id|><|start_header_id|>user<|end_header_id|>
If I tell, I might hurt them and lose the friendship. If I don't, I feel guilty keeping this secret.<|eot_id|><|start_header_id|>assistant<|end_header_id|>
You're weighing the duty of honesty against the potential for harm. There's also the question of your role and responsibility in this situation. Have you considered first approaching their partner about what you know?<|eot_id|><|start_header_id|>user<|end_header_id|>
I hadn't thought about that option. But wouldn't that be like enabling the cheating?<|eot_id|><|start_header_id|>assistant<|end_header_id|>
Not necessarily. It could give them a chance to come clean themselves, which might be less damaging to all relationships involved. However, you might want to set a deadline - if they don't tell your friend by a certain time, you will.<|eot_id|>""",

    # Tests understanding of paradoxical behavior and defense mechanisms
    """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a helpful AI assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>
Why do I push away people who try to get close to me?<|eot_id|><|start_header_id|>assistant<|end_header_id|>
This is a common experience that often relates to fear of vulnerability or past experiences. Can you remember when you first noticed this pattern?<|eot_id|><|start_header_id|>user<|end_header_id|>
I guess after my last relationship ended badly. It really hurt.<|eot_id|><|start_header_id|>assistant<|end_header_id|>
It sounds like you might be protecting yourself from experiencing that kind of pain again. While this is a natural response to hurt, it can also prevent new, potentially meaningful connections. What would feeling safe enough to let someone in look like to you?<|eot_id|>"""
]

long_conversations = [
    # Conversation 1: Technical Problem-Solving with Emotional Component
    """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a helpful AI assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>
I'm trying to learn programming but I keep failing. I've tried three different courses and I just can't get it. Maybe I'm just not smart enough.<|eot_id|><|start_header_id|>assistant<|end_header_id|>
Intelligence isn't the key factor in learning to program - everyone struggles at first. Could you tell me about one specific concept or problem that you found particularly challenging?<|eot_id|><|start_header_id|>user<|end_header_id|>
Well, I keep getting stuck with functions. I don't understand when to use them and how to make them work together. Like, I can write basic code, but when I need to split it into functions, I get lost.<|eot_id|><|start_header_id|>assistant<|end_header_id|>
Think of functions like recipes in a cookbook. Each recipe (function) does one specific thing. Let's start simple - could you share a piece of code you've written that you'd like to break into functions?<|eot_id|><|start_header_id|>user<|end_header_id|>
Here's something I wrote:
for i in range(10):
    num = i * 2
    if num > 10:
        print(f"{num} is large")
    else:
        print(f"{num} is small")<|eot_id|><|start_header_id|>assistant<|end_header_id|>
Great example! Let's break this into two functions. First, one function to classify the number, and another to process the range of numbers. Here's how:

def classify_number(num):
    if num > 10:
        return "large"
    else:
        return "small"

def process_numbers(max_range):
    for i in range(max_range):
        num = i * 2
        result = classify_number(num)
        print(f"{num} is {result}")

Now you can just call process_numbers(10)

See how each function has one specific job?<|eot_id|><|start_header_id|>user<|end_header_id|>
Oh, I think I get it. The first function just decides if it's large or small, and the second function does the loop? But why not just keep it all in one piece?<|eot_id|><|start_header_id|>assistant<|end_header_id|>
Exactly! The benefits of splitting it up are:
1. You can reuse classify_number() anywhere in your code
2. If you need to change what "large" means, you only change one place
3. It's easier to test each piece separately
4. The code becomes more readable

Want to try modifying the classify_number function to use a different threshold?<|eot_id|><|start_header_id|>user<|end_header_id|>
Yeah, let's say numbers over 15 are large instead of 10.<|eot_id|><|start_header_id|>assistant<|end_header_id|>
Perfect! Just change the first function to:

def classify_number(num):
    if num > 15:
        return "large"
    else:
        return "small"

The best part is, you don't need to change anything else in your code. This is one of the main benefits of functions - you can modify one piece without touching the rest.<|eot_id|><|start_header_id|>user<|end_header_id|>
That makes sense! I think I was making it more complicated in my head. Could we try another example?<|eot_id|><|start_header_id|>assistant<|end_header_id|>
Of course! Let's write a program that takes a name and gives a personalized greeting based on the time of day. What do you think the different functions might be for that?<|eot_id|>""",

    # Conversation 2: Complex Ethical Scenario with Multiple Stakeholders
    """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a helpful AI assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>
I'm a manager at a small company, and I just discovered that one of my best employees has been faking their college degree. They've been doing excellent work for two years. What should I do?<|eot_id|><|start_header_id|>assistant<|end_header_id|>
This is a complex situation that involves multiple considerations. Before discussing actions, could you tell me: is a college degree a strict requirement for their position according to company policy?<|eot_id|><|start_header_id|>user<|end_header_id|>
Yes, it's required. The job posting listed it as a requirement, and it's in our HR policies. But they're really good at their job and training someone new would be really difficult.<|eot_id|><|start_header_id|>assistant<|end_header_id|>
I understand the practical challenge. There are several stakeholders to consider here:
1. The employee and their livelihood
2. Your company's policies and integrity
3. Other employees who were honest about their credentials
4. Your own professional responsibilities

Have you considered discussing this directly with the employee first?<|eot_id|><|start_header_id|>user<|end_header_id|>
I'm worried about that conversation. What if they deny it? Or worse, what if they admit it and I'm then forced to fire them? Maybe I could just pretend I never found out?<|eot_id|><|start_header_id|>assistant<|end_header_id|>
Ignoring it might seem easier, but it could create bigger problems:
1. It puts you in a compromised position ethically
2. If discovered later, it could jeopardize your own position
3. It might set a precedent of dishonesty being acceptable
4. It could be unfair to other employees

A direct conversation, while difficult, allows for potential solutions. Would you like to discuss how to approach that conversation constructively?<|eot_id|><|start_header_id|>user<|end_header_id|>
Yes, please. I'm really nervous about it though.<|eot_id|><|start_header_id|>assistant<|end_header_id|>
Let's plan the conversation:
1. Start with their positive contributions
2. Express your concern about the discovery
3. Give them a chance to explain
4. Be clear about company policy
5. Possibly explore alternative solutions like:
   - Taking a leave to complete the degree
   - Moving to a position without the requirement
   - Creating a performance-based exception

Would you like to role-play the conversation?<|eot_id|><|start_header_id|>user<|end_header_id|>
Yes, that might help. How would you start it?<|eot_id|><|start_header_id|>assistant<|end_header_id|>
Try something like: "[Name], I've really appreciated your contributions to the team over the past two years. You've consistently shown excellent skills and work ethic. However, something has come to my attention that we need to discuss regarding your employment qualifications."

This opens the conversation positively while being direct. How do you think they might respond?<|eot_id|><|start_header_id|>user<|end_header_id|>
They might get defensive or scared. I would be. I feel terrible about this whole situation.<|eot_id|><|start_header_id|>assistant<|end_header_id|>
Your empathy is actually an asset here. You can acknowledge the difficulty while maintaining professional boundaries: "I understand this is a difficult conversation, and I value your contributions. I want to find a solution if possible, but we need to address this honestly first."

Remember, your role is to handle this ethically while being humane. Would you like to continue the role-play?<|eot_id|>""",

    # Conversation 3: Creative Project Development with Learning Elements
    """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a helpful AI assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>
I want to write a children's book about climate change, but I'm worried about making it too scary. Any advice?<|eot_id|><|start_header_id|>assistant<|end_header_id|>
That's a thoughtful concern. Children's books about serious topics need to balance awareness with hope. What age group are you writing for?<|eot_id|><|start_header_id|>user<|end_header_id|>
I'm thinking 7-9 year olds. Old enough to understand some complexity but still need a positive message.<|eot_id|><|start_header_id|>assistant<|end_header_id|>
Perfect age range. At 7-9, children can grasp cause and effect and are developing empathy. Consider structuring your story around:
1. A relatable protagonist
2. A specific, solvable problem
3. Positive actions that make a difference
4. Hope for the future

Would you like to brainstorm some possible story approaches?<|eot_id|><|start_header_id|>user<|end_header_id|>
Yes! I was thinking maybe something with animals? Kids love animals.<|eot_id|><|start_header_id|>assistant<|end_header_id|>
Great idea! Animals can make environmental messages more engaging and less threatening. We could have:
- A polar bear cub learning about their changing home
- A group of forest animals working together to clean their habitat
- A migrating bird discovering how different places are connected

Which approach interests you most?<|eot_id|><|start_header_id|>user<|end_header_id|>
I like the forest animals idea. It could show teamwork and community action.<|eot_id|><|start_header_id|>assistant<|end_header_id|>
Excellent choice! This could demonstrate both environmental stewardship and cooperation. Let's develop the basic elements:

1. Characters: Which forest animals should we include? Each could have different skills.
2. Problem: What issue are they facing?
3. Solution: How do they work together?
4. Message: What can kids learn and do?

Which should we develop first?<|eot_id|><|start_header_id|>user<|end_header_id|>
Let's start with the characters. I definitely want a wise owl, maybe a hardworking beaver, and some smaller animals too.<|eot_id|><|start_header_id|>assistant<|end_header_id|>
Great choices! Each animal could represent different approaches to problem-solving:
- Owl: The thoughtful planner
- Beaver: The practical builder
- Maybe add:
  - Squirrel: Quick and resourceful
  - Rabbit: Fast messenger
  - Fox: Clever strategist

Should we give them specific personalities or roles in the story?<|eot_id|><|start_header_id|>user<|end_header_id|>
Yes! The owl could be like a teacher but maybe a bit forgetful to make it funny. And the beaver could be super organized but needs to learn to be flexible.<|eot_id|><|start_header_id|>assistant<|end_header_id|>
Love those character quirks! They make the animals more relatable and add humor. They also create opportunities for character growth. How about:

- Owl: Wise but absent-minded Professor Hoot
- Beaver: Bob the meticulous builder
- Squirrel: Zippy the easily-distracted collector
- Rabbit: Flash the worried messenger
- Fox: Clever Claire the problem-solver

These personalities could create both funny moments and learning opportunities. Should we think about how they discover the environmental problem?<|eot_id|><|start_header_id|>user<|end_header_id|>
Maybe they notice their stream is getting polluted? That's something kids could relate to in their own communities.<|eot_id|><|start_header_id|>assistant<|end_header_id|>
Perfect! This works on multiple levels:
1. It's visible and understandable
2. Shows local impact of pollution
3. Has clear cause and effect
4. Offers opportunities for action

We could have each animal notice different aspects:
- Beaver: Can't build properly with trash in the water
- Owl: Notices fewer fish
- Squirrel: Finds plastic instead of nuts
- Rabbit: Worried about drinking water
- Fox: Discovers where the pollution is coming from

How should they start addressing the problem?<|eot_id|>"""
]

### Processing

In [ ]:
prompts = [*difficult_prompts, *additional_prompts, *self_awareness_prompts, *psychological_prompts, *conversations, *psychological_conversations, *long_conversations]
# Analyze prompts in batch
analysis_result_batch = analyze(prompts)
match analysis_result_batch:
    case Success(all_results):
        [visualize(single_result, storage=storage) for single_result in all_results]
    case Failure(error):
        raise RuntimeError(f"Batch analysis failed: {error}")

Calculating token association:  10%|█         | 4/39 [06:08<1:03:02, 108.08s/token]